In [2]:
%matplotlib tk
# --------------------------------------------------
#
# 2 stacks of averages
#
# --------------------------------------------------

import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from hmmlearn import hmm
import warnings
warnings.filterwarnings("ignore")


def boundaries(v):
    bounds = [0] + list(np.where(v[:-1] != v[1:])[0] + 1) + [len(v)-1]
    return [(bounds[i], bounds[i+1], v[bounds[i]]) for i in range(len(bounds)-1)]

def to_range(l, cmap):
    total = cmap.N
    idx = int((l[-1] / N) * total)
    return cmap(idx)

def train_model(x, n=3, m=10, _covar='diag', _iters=100):
    model = hmm.GMMHMM(n,m,covariance_type=_covar, n_iter=_iters)
    print('training: {}'.format(x.shape))
    model.fit(x)
    print('predicting: {}'.format(x.shape))
    z = model.predict(x)
    return z

def blur(x, n):
    if n % 2 == 0:
        raise ValueError('n must be odd, given: {}'.format(n))
    p = int((n-1) / 2)
    print('p = {}'.format(p))
    dims = x.shape[1]
    _x = np.pad(x,((p,p),(0,0)), mode='edge')
    _x = [np.convolve(_x[:,i], np.ones((n,))/n, mode='valid') for i in range(dims)]
    return np.stack(_x, axis=-1)



def plot_bounds(ax, x, z):
    Zb = boundaries(z)
    print('plotting: {}'.format(x.shape))
    event_labels = {i:i for i in range(N)}
    ax.plot(x[:,:-3], alpha=.7)
    ax.set_ylabel('gyroscope')
    ax.set_xlabel('time (milliseconds)')
    ax.set_ylim(-5000, 5000)
    # plot labels
    _cmap = plt.get_cmap('nipy_spectral')
    plots = {}
    for l in Zb:
        ev = event_labels[l[-1]]
        i = plt.axvspan(l[0], l[1], color=to_range(l, _cmap), alpha=0.5, label=ev)
        plots[ev] = i
    # legend
    legs = [plots[event_labels[i]] for i in sorted(event_labels)]
    nms = [event_labels[i] for i in sorted(event_labels)]
    plt.legend(legs, nms)


# params
DATASET = 'pills-s0'
BLUR = 5
S = 2
tiers = 4


# file and data
FILE = 'data/' + DATASET + '.npy'
X0 = np.load(FILE)


# Create Pyramids
X, B, D = [X0], [], []
for i in range(tiers):
    xi = X[i]
    # blur image
    x_blur = blur(xi,BLUR)
    print(xi.shape, x_blur.shape)
    B.append(x_blur)
    # take difference
    diff = xi - x_blur
    D.append(diff)
    # down-sample
    x_next = xi[::S]
    X.append(x_next)



fig, axes = plt.subplots(nrows=3, ncols=tiers, sharey='col', squeeze=False)
print('axes:', axes.shape)
# Plot
for i, ax in enumerate(axes.T):
    # original
    ax[0].plot(X[i])
    # blurred
    ax[1].plot(B[i])
    # difference
    ax[2].plot(D[i])

cols = ['original'] + ['x1/{}'.format(S ** col) for col in range(1, tiers)]
rows = ['original', 'blurred', 'difference']
for ax, col in zip(axes[0], cols):
    ax.set_title(col)
for ax, row in zip(axes[:,0], rows):
    ax.set_ylabel(row, rotation=0, size='large')    


# fig.tight_layout()
plt.show()

p = 2
(4506, 6) (4506, 6)
p = 2
(2253, 6) (2253, 6)
p = 2
(1127, 6) (1127, 6)
p = 2
(564, 6) (564, 6)
axes: (3, 4)
